<a href="https://colab.research.google.com/github/nsk-ai/RAG-Bootcamp-2025/blob/main/Reranking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Reranking

In [ ]:
pip install langchain langchain-community langchain-text-splitters langchain-groq chromadb sentence-transformers huggingface-hub


In [ ]:
from google.colab import userdata
import os

# Access the secret using userdata.get()
my_variable = userdata.get('GROQ_API_KEY')

# You can also set it as an environment variable for use with os.getenv()
os.environ['GROQ_API_KEY'] = my_variable

In [ ]:
# A quick standalone demonstration of how a cross-encoder works.
from sentence_transformers import CrossEncoder

# Load a pre-trained cross-encoder model
# This is a lightweight but effective model fine-tuned for semantic search.
reranker_model = CrossEncoder("cross-encoder/ms-marco-MiniLM-L6-v2")

# Let's create some query-document pairs to score
query = "What is the capital of France?"
documents = [
    "Paris is the capital of France.",
    "The Eiffel Tower is a famous landmark in Paris.",
    "Berlin is the capital of Germany.",
    "France is a country in Western Europe known for its cuisine and wine."
]

# The model expects a list of (query, document) pairs
sentence_pairs = [(query, doc) for doc in documents]

In [ ]:
sentence_pairs

[('What is the capital of France?', 'Paris is the capital of France.'),
 ('What is the capital of France?',
  'The Eiffel Tower is a famous landmark in Paris.'),
 ('What is the capital of France?', 'Berlin is the capital of Germany.'),
 ('What is the capital of France?',
  'France is a country in Western Europe known for its cuisine and wine.')]

In [ ]:
# Let's get the scores
# The output is a list of scores, one for each pair. Higher score = more relevant.
scores = reranker_model.predict(sentence_pairs)

In [ ]:
# Let's see the results
print("--- Manual Cross-Encoder Scoring ---")
for i in range(len(scores)):
    print(f"Score: {scores[i]:.4f}\t Document: {documents[i]}")

--- Manual Cross-Encoder Scoring ---
Score: 8.5007	 Document: Paris is the capital of France.
Score: -5.6306	 Document: The Eiffel Tower is a famous landmark in Paris.
Score: -2.5410	 Document: Berlin is the capital of Germany.
Score: -0.7809	 Document: France is a country in Western Europe known for its cuisine and wine.


**Putting It All Together in LangChain**

Now, let's build the full pipeline. We'll set up a basic retriever and then wrap it with our reranker.

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

# For the reranker
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

# For our demo RAG chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [ ]:
# --- 1. Setup Base Retriever (What we've learned before) ---

# Assume we have some text data
doc_text = """
The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France. It is named after the engineer Gustave Eiffel, whose company designed and built the tower.
The Louvre, or the Louvre Museum, is the world's most-visited museum, and a historic landmark in Paris, France.
The Arc de Triomphe is one of the most famous monuments in Paris, standing at the western end of the Champs-Élysées.
Paris is the capital and most populous city of France. The city is known for its art, fashion, gastronomy and culture.
Berlin, the capital of Germany, is known for its exceptional art scene and modern landmarks.
The Brandenburg Gate is a famous landmark in Berlin.
"""
with open("paris_facts.txt", "w") as f:
    f.write(doc_text)

In [ ]:
# Load and split the document
loader = TextLoader("paris_facts.txt")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
docs = text_splitter.split_documents(documents)

In [ ]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/msmarco-distilbert-dot-v5"
)
vector_store = Chroma.from_documents(docs, embeddings)

InvalidArgumentError: Collection expecting embedding with dimension of 384, got 768

In [ ]:
# This is our Stage 1 retriever. It will fetch 10 documents.
base_retriever = vector_store.as_retriever(search_kwargs={"k": 10})

In [ ]:
# --- 2. Initialize the Reranker ---

# Initialize LangChain's reranker compressor
# It uses the specified cross-encoder model
# top_n determines how many documents are returned after reranking
model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-base")
compressor = CrossEncoderReranker(model=model,
                                  top_n=3 # Return the top 3 most relevant documents
                                  )

In [ ]:
# --- 3. Create the Contextual Compression Retriever ---

# This is our Stage 2 retriever.
# It chains the base_retriever and the reranker.
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=base_retriever
)

In [ ]:
# --- 4. Use the Reranked Retriever ---

query = "What are some famous landmarks in Paris?"

In [ ]:
# Let's compare the results!

print("--- 4a. Results from Base Retriever (NO Reranking) ---")
base_retrieved_docs = base_retriever.invoke(query)
for i, doc in enumerate(base_retrieved_docs):
    print(f"Document {i+1}:\n{doc.page_content}\n")

In [ ]:
print("\n--- 4b. Results from Compression Retriever (WITH Reranking) ---")
reranked_docs = compression_retriever.invoke(query)
for i, doc in enumerate(reranked_docs):
    # The reranker adds a 'relevance_score' to the document metadata
    score = doc.metadata.get('relevance_score', 'N/A')
    # Check if the score is a number before formatting
    if isinstance(score, (int, float)):
        print(f"Document {i+1} (Score: {score:.4f}):\n{doc.page_content}\n")
    else:
        print(f"Document {i+1} (Score: {score}):\n{doc.page_content}\n")

In [ ]:
reranked_docs

### RAG Chain Example

In [ ]:
%pip install -qU langchain-community unstructured pdfminer pi_heif Pillow pypdf

In [ ]:
pip install "unstructured[pdf]"

In [ ]:
pip install pdfminer.six

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import OnlinePDFLoader


# For the reranker
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

# For our demo RAG chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [ ]:
# --- 1. Load a PDF from the Internet ---
pdf_url = "https://info.publicintelligence.net/MCIA-EthiopiaCultureGuide.pdf"
loader = OnlinePDFLoader(pdf_url)
documents = loader.load()

In [ ]:
# --- 2. Split the PDF into chunks ---
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(documents)

In [ ]:
# --- 3. Set up HuggingFace Embeddings and Vector Store ---
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_store = Chroma.from_documents(docs, embedding_model)

In [ ]:
# Base retriever (Stage 1)
base_retriever = vector_store.as_retriever(search_kwargs={"k": 10})

In [ ]:
# --- 4. Initialize the Reranker ---
model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-base")
compressor = CrossEncoderReranker(model=model,
                                  top_n=3 # Return the top 3 most relevant documents
                                  )

In [ ]:
# --- 5. Create Contextual Compression Retriever ---
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=base_retriever
)

In [ ]:
# --- 6. Compare the Results (Optional) ---
query = "What does the document say about Family in Ethiopia?"

print("\n--- Base Retriever Results ---")
base_docs = base_retriever.invoke(query)
for i, doc in enumerate(base_docs):
    print(f"Doc {i+1}:\n{doc.page_content[:300]}...\n")


--- Base Retriever Results ---
Doc 1:
Family ties remain significant even across vast distances. Ethiopi- ans living abroad are expected to send money home to their fam- ily. Unfortunately, rapid population growth combined with high poverty rates has taken a toll on Ethiopian family ties. Divorce is increasing as husbands find they can ...

Doc 2:
roles anD centers oF aUthority

local organization and leadership authority Traditionally, families and religious networks cared for the needy and passed cultural values to the next generation. These local net- works are still important and are often more influential than the national government in ...

Doc 3:
In general, Ethiopians have a positive view of U.S. citizens and culture. Older Ethiopians fondly recall the presence of Peace Corps volunteers during the reign of Haile Selassie, while young- er Ethiopians are proud to own the latest electronics and clothing from the United States.

identity and af...

Doc 4:
health concerns and acces

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [ ]:
print("\n--- Reranked Retriever Results ---")
reranked_docs = compression_retriever.invoke(query)
for i, doc in enumerate(reranked_docs):
    score = doc.metadata.get("relevance_score", "N/A")
    print(f"Doc {i+1} (Score: {score}):\n{doc.page_content[:300]}...\n")


--- Reranked Retriever Results ---


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Doc 1 (Score: N/A):
roles anD centers oF aUthority

local organization and leadership authority Traditionally, families and religious networks cared for the needy and passed cultural values to the next generation. These local net- works are still important and are often more influential than the national government in ...

Doc 2 (Score: N/A):
In general, Ethiopians have a positive view of U.S. citizens and culture. Older Ethiopians fondly recall the presence of Peace Corps volunteers during the reign of Haile Selassie, while young- er Ethiopians are proud to own the latest electronics and clothing from the United States.

identity and af...

Doc 3 (Score: N/A):
Family ties remain significant even across vast distances. Ethiopi- ans living abroad are expected to send money home to their fam- ily. Unfortunately, rapid population growth combined with high poverty rates has taken a toll on Ethiopian family ties. Divorce is increasing as husbands find they can ...



In [ ]:
# --- 7. Build a RAG Chain with Groq ---
from langchain_groq import ChatGroq

prompt_template = ChatPromptTemplate.from_template(
    "Answer the question using only the context below:\n\n{context}\n\nQuestion: {question}"
)

llm = ChatGroq(model_name="llama3-8b-8192", temperature=0)

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": compression_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

In [ ]:
print("\n--- Final Answer ---")
final_answer = rag_chain.invoke(query)
print(final_answer)


--- Final Answer ---


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


According to the document, in Ethiopia:

* Family is important to Ethiopians, and having children is an important part of social life, increasing a woman's prestige.
* On average, a woman has six children.
* Children are often called upon to support their elderly parents.
* The importance placed on extended family and the higher status for women who have large numbers of children leads to early marriage and large families.
* Family ties remain significant even across vast distances, with Ethiopians living abroad expected to send money home to their families.
* Unfortunately, rapid population growth combined with high poverty rates has taken a toll on Ethiopian family ties, leading to increasing divorce rates and single women moving to urban areas in search of work.
